In [1]:
import pandas as pd
import smtplib
import os
import json
import matplotlib.pyplot as plt
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email import encoders
from email.utils import make_msgid

In [2]:
# Configuración de comportamiento
ORIGEN = 'misiontic.noresponder@outlook.com'
ASUNTO = 'GRUPOS Y HORARIOS ASIGNADOS - CICLO 4'
SERVIDOR_SMTP = 'smtp-mail.outlook.com:587'
CORREO_TEST = False
ENVIO_A_PENDIENTES = False

if CORREO_TEST:
    RUTA = './INPUT/INFO_CURSOS/TEST_3.xlsx'
elif ENVIO_A_PENDIENTES:
    RUTA = './INPUT/INFO_CURSOS/ENVIOS_PENDIENTES_CREDENCIALES.xlsx'
else:
    RUTA = './INPUT/INFO_CURSOS/LISTA_CREDENCIALES_FORMADORES.xlsx'

COLUMNA_CORREO = 'CORREO_DOCENTE'
COLUMNAS = [
    'ASIGNATURA',
    'GRUPO',
    'DIA',
    'HORA_INICIAL_',
    'HORA_FINAL_',
    'CUENTA_ZOOM_UIS_DEFINITIVA_2022',    
    'CONTRASEÑA_ZOOM',
    'ENLACE_ZOOM_CLASES_SINCRÓNICAS',
]

In [3]:
MSG_HEAD = """<html><body><p>Bucaramanga, 29 de septiembre de 2022<br>
Respetado profesional<br>
<b>{formador}</b><br>
Programa Misión TIC 2022 - Ciclo 3</p>

"""
MSG_BODY = """<p>Dado que este lunes 3 de octubre inicia el ciclo 4, se le comparten sus grupos asignados y sus horarios. Tenga en cuenta que la hora va en <u><b>FORMATO DE 24 HORAS</u></b>:<br></p>

"""

MSG_FOOTER = """<p>ESTE CORREO SE ENVIÓ DE FORMA AUTOMÁTICA, NO RESPONDA A ESTA DIRECCIÓN O CORREO. Cualquier información, contacte a misiontic.monitor@uis.edu.co o rectoria.misiontic@uis.edu.co.</p>
<p>Tecnología diseñada por <a href="https://nuwebs.com.co">Nuwebs</a></p>
<p>Bucaramanga, Colombia. +57 3184301032</p>
</body></html>"""

In [4]:
f = open('CREDENCIALES.json')
CREDENCIALES = json.load(f)
f.close()
datos = pd.read_excel(RUTA, engine = 'openpyxl')

In [5]:
datos

,ASIGNATURA,GRUPO,DIA,HORA_INICIAL_,HORA_FINAL_,DOCENTE,NOMBRE_DOCENTE,CORREO_DOCENTE,CORREO_INSTITUCIONAL_DOCENTE,TELEFONO_DOCENTE,CUENTA_ZOOM_UIS_DEFINITIVA_2022,ENLACE_ZOOM_CLASES_SINCRÓNICAS,CONTRASEÑA_ZOOM
0,DESARROLLO DE APLICACIONES WEB,U14,LUNES A VIERNES,14:00:00,16:00:00,1098653387,JHON JAIRO CORTES PAREDES,jhonjairo.cortesp@gmail.com,misiontic.formador64@uis.edu.co,3177891483,zoom848@uis.edu.co,https://renata.zoom.us/j/86569929917,1Qt2y51q98
1,DESARROLLO DE APLICACIONES WEB,U30,LUNES A VIERNES,20:00:00,22:00:00,91512795,EDDINSON JULIAN CARRILLO SISA,edjuca2@gmail.com,misiontic.formador39@uis.edu.co,3153092059,zoom819@uis.edu.co,https://renata.zoom.us/j/89041173266,dt4v36F4DY
2,DESARROLLO DE APLICACIONES WEB,U16,LUNES A VIERNES,16:00:00,18:00:00,73167775,CARLOS ANDRES GUERRERO ALARCON,anguerrco@msn.com,misiontic.formador68@uis.edu.co,3017900437,zoom826@uis.edu.co,https://renata.zoom.us/j/84854225679,257382EqDY
3,DESARROLLO DE APLICACIONES WEB,U36,LUNES A VIERNES,20:00:00,22:00:00,1072652370,DANIEL RINCON LEON,drincon.89@gmail.com,misiontic.formador7@uis.edu.co,3057044609,zoom829@uis.edu.co,https://renata.zoom.us/j/88688339512,j6H3tZ5JDY
4,DESARROLLO DE APLICACIONES WEB,U33,LUNES A VIERNES,20:00:00,22:00:00,18973252,LUIS ALBERTO SAURITH ALVARADO,sauriths@gmail.com,misiontic.formador79@uis.edu.co,3002232559,zoom851@uis.edu.co,https://renata.zoom.us/j/87409868189,W29%Cj2kdy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,COACHING IV,X4,Jueves,6:00:00,7:00:00,1065880115,VIVIAN CAROLINA PEREZ ALMANZA,ps.vivianperez@gmail.com,misiontic.coach6@uis.edu.co,3133929166,zoom828@uis.edu.co,https://renata.zoom.us/j/82440987906,4qN3Q6t7DY
157,COACHING IV,X14,Miércoles,10:00:00,11:00:00,1005552978,PAOLA ANDREA FLOREZ GOMEZ,paitoflogo@gmail.com,misiontic.coach10@uis.edu.co,3203338837,zoom872@uis.edu.co,https://renata.zoom.us/j/81119799064,78Mwc5@MDY
158,COACHING IV,X52,Lunes,21:00:00,22:00:00,1098740925,PAOLA ANDREA SANTOS ORDONES,paola08.psi@gmail.com,misiontic.coach7@uis.edu.co,3166301406,zoom871@uis.edu.co,https://renata.zoom.us/j/81272944088,p6h5Dv31DY
159,COACHING IV,X36,Jueves,17:00:00,18:00:00,1098740925,PAOLA ANDREA SANTOS ORDONES,paola08.psi@gmail.com,misiontic.coach7@uis.edu.co,3166301406,zoom871@uis.edu.co,https://renata.zoom.us/j/81272944088,p6h5Dv31DY


In [6]:
datos['HORA_INICIAL_'] = datos['HORA_INICIAL_'].astype(str)
datos['HORA_FINAL_'] = datos['HORA_FINAL_'].astype(str)

In [7]:
def generarCorreo(origen, destino, asunto):
    multipart = MIMEMultipart('related') # Posible quitar related
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart['X-Priority'] = '2'
    multipart.preamble = '====================================================='
    
    return multipart

def getCuerpoCorreo(multipart, formador, df): 
    msgAlternative = MIMEMultipart('alternative') #Posible quitar esto
    multipart.attach(msgAlternative)
    
    cuerpoAdicional = '<table border="1"><tbody><tr>'
    for columna in COLUMNAS:
        cuerpoAdicional += '<th>' + columna + '</th>'
    cuerpoAdicional += '</tr>'
    for index, row in df.iterrows():
        cuerpoAdicional += '<tr>'
        for columna in COLUMNAS:            
            cuerpoAdicional += '<td>' + row[columna] + '</td>'
        cuerpoAdicional += '</tr>'        
    cuerpoAdicional += '</tbody></table>'
    cuerpoAdicional += '<p><b>Nota:</b> Si no puede visualizar la tabla, sus credenciales igualmente están en el archivo adjunto a este correo</p>'
    cuerpo = MSG_HEAD.format(formador = formador) + MSG_BODY + cuerpoAdicional + MSG_FOOTER
    cuerpo = MIMEText(cuerpo, 'html', 'utf-8')
    msgAlternative.attach(cuerpo)
    return multipart, cuerpo

def getAdjuntos(multipart, imgs = [], adjuntos = []):
    for img in imgs:
        multipart.attach(img)
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

In [8]:
enviados = datos.copy()
destinos = datos[COLUMNA_CORREO].unique()
servidor = smtplib.SMTP(SERVIDOR_SMTP)
servidor.starttls()
servidor.login(CREDENCIALES['USUARIO'], CREDENCIALES['PASS'])
cont = 1
for destino in destinos:
    dfD = datos[datos[COLUMNA_CORREO] == destino]
    nombreFormador = dfD['NOMBRE_DOCENTE'].iloc[0]
    adjuntos = []
    nombreArchivo = 'GRUPOS_ASIGNADOS_' + nombreFormador.replace(' ', '_')
    ruta = './Temp/' + nombreArchivo + '.xlsx'
    dfD[COLUMNAS].to_excel(ruta, index = False)
    adjuntos.append(ruta)
    
    base = generarCorreo(ORIGEN, destino, ASUNTO)
    base, contenido = getCuerpoCorreo(base, nombreFormador, dfD)
    base = getAdjuntos(base, [], adjuntos)
    try:        
        servidor.sendmail(ORIGEN, destino, base.as_string())
        enviados = enviados[enviados[COLUMNA_CORREO] != destino]
    except:
        enviados.to_excel('./INPUT/INFO_CURSOS/ENVIOS_PENDIENTES_CREDENCIALES.xlsx', index=False)
        print ('Ocurrió un error al enviar el correo a', destino)
        break
    print (cont, 'Enviado a', nombreFormador, destino)
    cont += 1
servidor.quit()

1 Enviado a JHON JAIRO CORTES PAREDES jhonjairo.cortesp@gmail.com
2 Enviado a EDDINSON JULIAN CARRILLO SISA edjuca2@gmail.com
3 Enviado a CARLOS ANDRES GUERRERO ALARCON anguerrco@msn.com
4 Enviado a DANIEL RINCON LEON drincon.89@gmail.com
5 Enviado a LUIS ALBERTO SAURITH ALVARADO sauriths@gmail.com
6 Enviado a CARLOS HORACIO GARCIA CASTRILLON carloshgcastrillon@gmail.com
7 Enviado a CARLOS SANTIAGO BENITEZ MALAVER santiago9607b@gmail.com
8 Enviado a CARLOS ARTURO PARRA ORTEGA carapa2000@hotmail.com
9 Enviado a SERGIO NICOLAS PINILLA CANON pinilla.nicolas10@gmail.com
10 Enviado a DIEGO FERNANDO MANTILLA VEGA diego.mantilla.trabajo@gmail.com; dfmco999@gmail.com
11 Enviado a SILVIA JULIANA  MORENO ROA julianamroa@gmail.com
12 Enviado a CARLOS ANDRES PALMA SUAREZ carlospalma_sistemas@hotmail.com
13 Enviado a JAHIR AUGUSTO SAAVEDRA GARCIA jahir.saavedra@gmail.com
14 Enviado a GUILLERMO BEJARANO REYES gbejarano@gmail.com
15 Enviado a ALEX DAVID MORALES ACOSTA alexmo14@gmail.com
16 Enviado a 

(221, b'2.0.0 Service closing transmission channel')